In [1]:
import mysql.connector
import json
import time
def get_connection():
    return mysql.connector.connect(
        host="kadman-database.c8boumioc8hx.us-east-1.rds.amazonaws.com",
        user="admin",
        password="12345678",
        database="kadmandb",
        port=3306,
        charset='utf8mb4'
    )

def wait_for_db_connection():
    while True:
        try:
            conn = get_connection()
            print("✅ Successfully connected to MySQL database")
            return conn
        except Exception as e:
            print(f"❌ Failed to connect to DB: {e}. Retrying in 1 seconds...")
            time.sleep(1)

In [2]:
con = wait_for_db_connection()
cursor = con.cursor()


✅ Successfully connected to MySQL database


In [ ]:
# Query all vendor data
cursor.execute("ALTER TABLE vendors DROP COLUMN lineID")
rows = cursor.fetchall()
print(rows)

In [16]:
cursor.execute("ALTER TABLE vendors ADD COLUMN UserProfile VARCHAR(20)")
con.commit()
cursor.close()
con.close()

In [ ]:
cursor = con.cursor()

# Replace 'your_table_name' with the actual table name
cursor.execute("SELECT * FROM vendors")

columns = cursor.fetchall()

print(columns)



[(1, None, 'อิงสร้อยผ้าโพก', None, '["เครื่องประดับ"]'), (2, None, 'โอเสื้อยืด,เชิ้ต,แว่นตา', None, '["new bagde"]'), (3, None, 'ไอซ์ stronger.store25 เสื้อยืดแจ็คเก็ต', None, '["เสื้อผ้า"]'), (4, None, 'พี่ตั้มหอยทากเสื้อยืดของเล่น', None, '["เสื้อผ้า", "ของเล่น"]'), (5, None, 'แม่ต่ายเสื้อยืด', None, '["เสื้อผ้า"]'), (6, None, 'แม่ต่ายแขนยาว', None, '["เสื้อผ้า"]'), (7, None, 'โอมเพี้ยงเสื้อยืด', None, '["เสื้อผ้า"]'), (8, None, 'พี่เบนซ์กระเป๋า', None, '["กระเป๋า"]'), (9, None, 'ฟ่าง y2k', None, '["เสื้อผ้า", "y2k"]'), (10, None, 'อาย daffodil.3store เสื้อฮิปปี้', None, '["เสื้อผ้า"]'), (11, None, 'บับเบิ้ลเสื้อยืด เนคไท ผ้าเช็ดหน้า', None, '["เสื้อผ้า"]'), (12, None, 'พี่อาร์มเสื้อกีฬา', None, '["เสื้อผ้า"]'), (13, None, 'พี่ฮัทถุงเท้า', None, '["ถุงเท้า"]'), (14, None, 'พี่อ๋องแจ็คเก็ต', None, '["เสื้อผ้า"]'), (15, None, 'พี่วุฒิโปสเตอร์', None, '["โปสเตอร์"]'), (16, None, 'พี่พรฮิปปี้แฟรี่ y2k', None, '["เสื้อผ้า", "y2k"]'), (17, None, 'พี่เอ็ม & ครูมายัง y2k', None, '["เสื้อผ้า"

: 

In [54]:
conn = wait_for_db_connection()

try:
    cursor
except NameError:
    cursor = None

if cursor is not None:
    try:
        # fetch all unread results if any
        cursor.fetchall()
    except Exception:
        # ignore if no results or error
        pass
    cursor.close()
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS waiting_vendors")
print("✅ Table 'waiting_vendors' dropped successfully.")
cursor.close()
conn.commit()
cursor.close()


✅ Successfully connected to MySQL database
✅ Table 'waiting_vendors' dropped successfully.


False

In [55]:
def create_waiting_vendors_table():
    conn = get_connection()
    cursor = conn.cursor()

    create_table_sql = """
    CREATE TABLE IF NOT EXISTS waiting_vendors (
        LineID VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        UserProfile VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        PRIMARY KEY (LineID)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """

    try:
        cursor.execute(create_table_sql)
        print("✅ Table 'waiting_vendors' created successfully.")
    except mysql.connector.Error as err:
        print(f"❌ Error creating table: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
create_waiting_vendors_table()


✅ Table 'waiting_vendors' created successfully.


In [ ]:
def show_all_rows():
    conn = get_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT * FROM layouts")
        rows = cursor.fetchall()
        if rows:
            print(f"Rows in 'waiting_vendors' ({len(rows)} rows):")
            for row in rows:
                print(row)
        else:
            print("No rows found in 'waiting_vendors' table.")
    except mysql.connector.Error as err:
        print(f"Error fetching rows: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
show_all_rows()

No rows found in 'waiting_vendors' table.


: 

In [ ]:
conn = wait_for_db_connection()
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS real_vendors (
    vendorsID INTEGER PRIMARY KEY AUTO_INCREMENT,
    shopname VARCHAR(255) NOT NULL,
    phone_number VARCHAR(20) NOT NULL,
    badges JSON,             -- MySQL supports a JSON data type in recent versions
    UserProfile VARCHAR(1000) NOT NULL
);
''')

✅ Successfully connected to MySQL database


In [ ]:
sql = 'SHOW COLUMNS FROM kadmandb.columns'